# Exploration du jeu de données expert
Ce notebook explore le fichier `data/expert_dataset.npz` généré par le solveur optimal du Klondike. Il affiche quelques informations et visualisations de base sur les transitions enregistrées.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.decomposition import PCA

DATA_PATH = 'data/expert_dataset.npz'
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"Le fichier {DATA_PATH} est introuvable. Générez-le d'abord avec generate_expert_dataset.py")

data = np.load(DATA_PATH, allow_pickle=True)


## Taille et formes des tableaux
Les différentes clés du fichier ainsi que leurs formes et types sont listées ci-dessous.

In [ ]:
for key in data.files:
    arr = data[key]
    print(f"{key}: shape={arr.shape}, dtype={arr.dtype}")
print(f"Nombre total de transitions: {len(data[list(data.files)[0]])}")


## Distribution des intentions
Si le tableau `intentions` est présent, on trace la fréquence de chaque intention rencontrée.

In [ ]:
if 'intentions' in data:
    intentions = data['intentions']
    counts = Counter(intentions)
    labels, values = zip(*counts.items())
    plt.figure(figsize=(6,4))
    plt.bar(labels, values)
    plt.title('Distribution des intentions')
    plt.ylabel('Occurrences')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
else:
    print('Aucune intention présente dans le dataset.')


## Projection PCA des observations
Les observations (vecteurs de 156 features) sont projetées en 2D à l'aide d'une analyse en composantes principales pour visualiser leur répartition.

In [ ]:
observations = data['observations']
# échantillonner pour accélérer si le jeu de données est volumineux
sample_size = min(5000, len(observations))
subset = observations[:sample_size]
pca = PCA(n_components=2)
proj = pca.fit_transform(subset)
plt.figure(figsize=(6,6))
plt.scatter(proj[:,0], proj[:,1], s=5, alpha=0.5)
plt.title('Projection PCA des observations')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()


## Séquence d'actions d'une partie aléatoire
À partir du tableau `dones`, on découpe le dataset en parties. On sélectionne une partie au hasard et on affiche la séquence d'actions correspondante.

In [ ]:
actions = data['actions']
dones = data['dones']
# indices où une partie se termine
end_indices = np.where(dones)[0]
start_indices = np.concatenate(([0], end_indices[:-1] + 1))
episodes = list(zip(start_indices, end_indices + 1))

if episodes:
    import random
    start, end = random.choice(episodes)
    seq = actions[start:end]
    plt.figure(figsize=(8,3))
    plt.plot(range(len(seq)), seq, marker='o')
    plt.title('Actions pour une partie aléatoire')
    plt.xlabel('Étape')
    plt.ylabel("Indice d'action")
    plt.grid(True)
    plt.show()
else:
    print('Impossible de déterminer les parties depuis les données.')
